# Получение данных

In [1]:
!git clone https://github.com/Derinhelm/llm_tokenization

Cloning into 'llm_tokenization'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 32 (delta 1), reused 1 (delta 1), pack-reused 28 (from 1)
Receiving objects: 100% (32/32), 33.03 MiB | 7.53 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (11/11), done.


In [2]:
import sys
sys.path.append('/content/llm_tokenization')

In [3]:
from data_classes import ConllEntry, Sentence

In [4]:
import pickle

with open('/content/llm_tokenization/treebank_test_sets/treebank_data.pickle', 'rb') as f:
    treebanks = pickle.load(f)

In [5]:
treebank_names = list(treebanks.keys())
treebank_names

['gsd', 'pud', 'syntagrus', 'poetry', 'taiga']

# Создание набора подтокенов

In [6]:
from transformers import AutoTokenizer

model_names = [ "Qwen/Qwen3-4B", "RefalMachine/RuadaptQwen3-4B-Instruct",
                "Qwen/Qwen3-32B", "t-tech/T-pro-it-2.0"
                #"Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24", "StoneLabs/Mistral-Nemo-Instruct-2407",
                #"IlyaGusev/saiga_nemo_12b", "natong19/Mistral-Nemo-Instruct-2407-abliterated",
                #"IlyaGusev/saiga_llama3_8b", "NousResearch/Meta-Llama-3-8B-Instruct",
                #"IlyaGusev/saiga_mistral_7b_lora", "Open-Orca/Mistral-7B-OpenOrca",
                #"IlyaGusev/saiga2_7b_lora", "NousResearch/Llama-2-7b-hf",
              ]

In [7]:
tokenizers = {}
for m_name in model_names:
    m_title = m_name.split("/")[1].replace("-", "_")
    tokenizers[m_title] = AutoTokenizer.from_pretrained(m_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

In [8]:
model_titles = list(tokenizers.keys())
model_titles

['Qwen3_4B', 'RuadaptQwen3_4B_Instruct', 'Qwen3_32B', 'T_pro_it_2.0']

In [9]:
def create_inputs(tokenizer_param, text, apply_chat_template=True):
    if apply_chat_template:
      messages = [{"role": "user", "content": f"{text}"}]
      text = tokenizer_param.apply_chat_template(
            messages, tokenize=False,
            add_generation_prompt=False)
    inputs = tokenizer_param(text, return_offsets_mapping=True, return_tensors='pt')
    subtoken_texts = [tokenizer_param.decode(t) for t in inputs['input_ids'][0]]

    big_subtokens = []
    amount = 1
    id_list = inputs['input_ids'][0]
    while len(id_list) != 0:
      s1 = tokenizer_param.decode(id_list[:amount])
      if any([ord(symb) == 65533 for symb in s1]):
        amount += 1
      else:
        big_subtokens.append((s1, amount))
        id_list = id_list[amount:]
        amount = 1
    return (inputs, subtoken_texts, big_subtokens)

In [10]:
sent_subtokens = {}
for m_title in model_titles:
  apply_chat_template = True
  #if m_title in {"saiga_mistral_7b_lora", "Mistral_7B_OpenOrca", "saiga2_7b_lora", "Llama_2_7b_hf" }:
  #  apply_chat_template = False
  sent_subtokens[m_title] = {}
  for tr in treebank_names:
    sent_subtokens[m_title][tr] = {}
    for sent in treebanks[tr]:
      sent_subtokens[m_title][tr][sent.sent_id] = create_inputs(tokenizers[m_title],
          sent.text.replace('\xa0', " "), apply_chat_template) # Замена неразрывного пробела

In [11]:
for m_title in model_titles:
  with open(f'/content/sent_subtokens_{m_title}.pickle', 'wb') as f:
    pickle.dump(sent_subtokens[m_title], f)

In [12]:
from google.colab import files

for m_title in model_titles:
  files.download(f'/content/sent_subtokens_{m_title}.pickle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>